In [1]:
!wget https://production-media.paperswithcode.com/about/papers-with-abstracts.json.gz
!wget https://production-media.paperswithcode.com/about/links-between-papers-and-code.json.gz

--2022-03-27 23:18:05--  https://production-media.paperswithcode.com/about/papers-with-abstracts.json.gz
Resolving production-media.paperswithcode.com (production-media.paperswithcode.com)... 104.26.13.155, 172.67.73.69, 104.26.12.155, ...
Connecting to production-media.paperswithcode.com (production-media.paperswithcode.com)|104.26.13.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 212474797 (203M) [binary/octet-stream]
Saving to: ‘papers-with-abstracts.json.gz’

papers-with-abstrac 100%[===================>] 202.63M  55.8MB/s    in 3.7s    

2022-03-27 23:18:09 (54.9 MB/s) - ‘papers-with-abstracts.json.gz’ saved [212474797/212474797]

--2022-03-27 23:18:09--  https://production-media.paperswithcode.com/about/links-between-papers-and-code.json.gz
Resolving production-media.paperswithcode.com (production-media.paperswithcode.com)... 104.26.13.155, 172.67.73.69, 104.26.12.155, ...
Connecting to production-media.paperswithcode.com (production-media.paperswit

In [21]:
import os
import json
import gzip
import pandas as pd

In [6]:
with gzip.open("papers-with-abstracts.json.gz", "rb") as f:
    papersinfo = json.loads(f.read().decode("ascii"))
with gzip.open("links-between-papers-and-code.json.gz", "rb") as f:
    linksinfo = json.loads(f.read().decode("ascii"))

In [7]:
papersinfo[0]

{'abstract': 'Can evolving networks be inferred and modeled without directly observing\ntheir nodes and edges? In many applications, the edges of a dynamic network\nmight not be observed, but one can observe the dynamics of stochastic cascading\nprocesses (e.g., information diffusion, virus propagation) occurring over the\nunobserved network. While there have been efforts to infer networks based on\nsuch data, providing a generative probabilistic model that is able to identify\nthe underlying time-varying network remains an open question. Here we consider\nthe problem of inferring generative dynamic network models based on network\ncascade diffusion data. We propose a novel framework for providing a\nnon-parametric dynamic network model--based on a mixture of coupled\nhierarchical Dirichlet processes-- based on data capturing cascade node\ninfection times. Our approach allows us to infer the evolving community\nstructure in networks and to obtain an explicit predictive distribution ove

In [11]:
linksinfo[0]

{'framework': 'tf',
 'is_official': False,
 'mentioned_in_github': False,
 'mentioned_in_paper': False,
 'paper_arxiv_id': None,
 'paper_title': 'Automatic Post-Editing of Machine Translation: A Neural Programmer-Interpreter Approach',
 'paper_url': 'https://paperswithcode.com/paper/automatic-post-editing-of-machine-translation',
 'paper_url_abs': 'https://aclanthology.org/D18-1341',
 'paper_url_pdf': 'https://aclanthology.org/D18-1341.pdf',
 'repo_url': 'https://github.com/trangvu/ape-npi'}

In [26]:
paperlinks = dict()

In [27]:
for paperinfo in papersinfo:
    title = paperinfo['title']
    pdfurl = paperinfo['url_pdf']
    paperlinks[title] = [pdfurl]

In [28]:
for linkinfo in linksinfo:
    title = linkinfo['paper_title']
    repourl = linkinfo['repo_url']
    try:
        links = paperlinks[title]
        links.append(repourl)
        paperlinks[title] = links
    except:
        continue

In [29]:
paperswithcode = 0
paperswithoutcode = 0
paperswithoutlink = 0
paperswithmultiplecode = 0
keystoremove = []
for title in paperlinks:
    links = paperlinks[title]
    if len(links) == 2:
        paperswithcode += 1
    elif len(links) == 1:
        paperswithoutcode += 1
        keystoremove.append(title)
    elif len(links) == 0 or links[0] == '':
        paperswithoutlink += 1
        keystoremove.append(title)
    else:
        paperswithmultiplecode += 1
        keystoremove.append(title)

print('Papers with code: ', paperswithcode)
print('Papers without code: ', paperswithoutcode)
print('Papers with multiple code links: ', paperswithmultiplecode)
print('Papers without link: ', paperswithoutlink)

Papers with code:  54055
Papers without code:  216895
Papers with multiple code links:  13004
Papers without link:  0


In [30]:
for title in keystoremove:
    del paperlinks[title]

In [36]:
df = pd.DataFrame.from_dict(paperlinks, orient='index', columns=['link', 'code'])
df.index.name = 'paper'
df.head()

,link,code
paper,,
Temporal coherence-based self-supervised learning for laparoscopic workflow analysis,http://arxiv.org/pdf/1806.06811v2.pdf,https://gitlab.com/nct_tso_public/pretrain_tc
BinGAN: Learning Compact Binary Descriptors with a Regularized GAN,http://arxiv.org/pdf/1806.06778v5.pdf,https://github.com/maciejzieba/binGAN
Multiscale Fisher's Independence Test for Multivariate Dependence,https://arxiv.org/pdf/1806.06777v7.pdf,https://github.com/MaStatLab/multiFit
POTs: Protective Optimization Technologies,https://arxiv.org/pdf/1806.02711v6.pdf,https://github.com/spring-epfl/pots
Surface Networks,http://arxiv.org/pdf/1705.10819v2.pdf,https://github.com/jiangzhongshi/SurfaceNetworks


In [37]:
df.to_csv('paper-code-links.csv')